In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
# Newton's optimization method for multivariate function in tensorflow
def compute_hessian(fn, vars):
    grad = lambda f, v: tf.gradients(f, v)[0]
    mat = []
    for v1 in vars:
        temp = []
        for v2 in vars:
            #temp.append(tf.gradients(tf.gradients(fn, v2)[0], v1)[0])
            temp.append( grad(grad(fn, v2), v1) )
        temp = [tf.constant(0, dtype=tf.float32) if t == None else t for t in temp]
        temp = tf.stack(temp)
        mat.append(temp)
    mat = tf.stack(mat)
    return mat

def compute_grads(fn, vars):
    grads = []
    for v in vars:
        grads.append(tf.gradients(fn, v)[0])
    return tf.reshape(tf.stack(grads), shape=[len(vars), -1])

def optimize(all_variables, update, alpha=tf.constant(1.0, dtype=tf.float32)):
    optmize_variables = []
    for i in range(len(all_variables)):
        optmize_variables.append(all_variables[i].assign(all_variables[i] - alpha * tf.squeeze(update[i])))
    return tf.stack(optmize_variables)

In [ ]:
x = tf.Variable(np.random.random_sample(), dtype=tf.float32)
y = tf.Variable(np.random.random_sample(), dtype=tf.float32)

#f = tf.constant(0.5, dtype=tf.float32) * tf.pow(x, 2) + tf.constant(2.5, dtype=tf.float32) * tf.pow(y, 2)
f = tf.pow(x, 2) + tf.constant(2.0)*x*y + tf.constant(3.0)*tf.pow(y, 2) + tf.constant(4.0)*x + tf.constant(5.0)*y + tf.constant(6.0)
all_variables = [x, y]

hessian = compute_hessian(f, all_variables)
hessian_inv = tf.matrix_inverse(hessian)
first_order_grad = compute_grads(f, all_variables)
second_order_grad = tf.unstack(tf.matmul(hessian_inv, first_order_grad))

## first order optimization

In [ ]:
optimize_op = optimize(all_variables, first_order_grad, tf.constant(0.2))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

func = np.inf
for i in range(30):
    prev = func
    v1, v2, func = sess.run([x, y, f])
    print(v1, v2, func)
    sess.run(optimize_op)

## second order optimization

In [ ]:
optimize_op = optimize(all_variables, second_order_grad, tf.constant(0.9))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

func = np.inf
for i in range(10):
    prev = func
    v1, v2, func = sess.run([x, y, f])
    print(v1, v2, func)
    sess.run(optimize_op)

In [ ]:
optimize_op = optimize(all_variables, second_order_grad, tf.constant(1.0))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

func = np.inf
for i in range(10):
    prev = func
    v1, v2, func = sess.run([x, y, f])
    print(v1, v2, func)
    sess.run(optimize_op)